In [25]:
import numpy as np
def one_hot(sequence):
    nucl_list = ['A', 'T', 'C', 'G']
    nucl_dic = dict((int, nucl) for nucl, int in enumerate(nucl_list))
    _seq_data = [nucl_dic[base] for base in sequence]
    result = np.zeros((len(sequence), 4))
    for _, base in enumerate(_seq_data):
        result[_, base] = 1
    return(result)

In [26]:
one_hot("ATCG")

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [59]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import numpy as np

# convert sequence to matrix
def one_hot(sequence):
    nucl_list = ['A', 'T', 'C', 'G']
    nucl_dic = dict((int, nucl) for nucl, int in enumerate(nucl_list))
    _seq_data = [nucl_dic[base] for base in sequence]
    result = np.zeros((len(sequence), 4))
    for _, base in enumerate(_seq_data):
        result[_, base] = 1
    return(result)

# cut off sequence to 2k segment
def seq2onehot(sequence_file):

    for seq_record in SeqIO.parse(sequence_file, 'fasta'):
        # virus 2k DNA sequence segments
        new_record = []
        # get sequence data
        seq = seq_record.seq
        # remove N
        seq = str(seq).replace('N', '')
        
        for i in range(0, len(seq), 500):
            if (i + 2000) > len(seq):
                new_seq = seq[-2000:]
                record = SeqRecord(new_seq, id=seq_record.id)
                new_record.append(record)
                break
            new_seq = seq[i:i+2000]
            record = SeqRecord(new_seq, id=seq_record.id)
            new_record.append(record)
           
        seq_data = np.array([one_hot(segment) for segment in new_record])
        return seq_data
        

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# define CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__() # 继承父类nn.Module
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        return x

In [1]:
data_path = '/home/ouconstand/data/Virus_Host/seq_data/sequence_0.fasta'

['x', 'x', 'x', 'x']

In [60]:
train = seq2onehot(data_path)

In [63]:
train.shape

(56, 2000, 4)

In [46]:
len(train)

1